In [51]:
import nltk
import numpy as np
import random
import string
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

***Definición de Corpus***

In [52]:
from google.colab import files
# El comando abrirá un cuadro de diálogo para que yo pueda subir el archivo
uploaded = files.upload()

Saving Corpus_crucero.txt to Corpus_crucero (5).txt


In [53]:
f = open("Corpus_crucero.txt", errors='ignore')
raw = f.read()
print(raw)

Para saber cmo vestir, la estacin del ao y la temperatura media de la zona marcan siempre la vestimenta ms adecuada para los cruceros, aunque los cruceros tienden a ser cada vez ms informales sobre todo durante el da.

El programa del da le indicar siempre la ropa ms adecuada.

Respecto al calzado, en tierra necesitars zapatos cmodos para caminar, as como sombrero y proteccin solar (en verano).

Las propinas son una norma generalizada, obligatorias, en todos los barcos de crucero, de hecho, forman parte del sueldo del personal y forman parte del llamado servicio.

Dependiendo de la naviera, para embarcar es conveniente llegar al embarque 3 o 4 horas antes de la hora de zarpar, normalmente los embarques suelen ser al medioda y casi todas las compaas de cruceros ofrecen un buffet para comer.

La comida es un punto fuerte, los cruceros son una oportunidad nica para poder degustar una amplia gama de alta calidad, con una diversidad de platos tpicos de todas las partes del mundo, ya sea con

***PreProcesamiento del texto con NLTK corpus***

In [55]:
raw = raw.lower() # lo convierte en minúscula
sent_tokens = nltk.sent_tokenize(raw) # Convierte al Corpues en una lista de sentencias
word_tokens = nltk.word_tokenize(raw) # Covierte el Corpus  a una lista de palabras
lemmer = nltk.stem.WordNetLemmatizer()

# WordNet diccionario semántico incluido en NLTK
def LemTokens(tokens):
  return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
  return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

***Preprocesamiento del texto + 3 evaluar similitud mensaje usuario - Corpus***

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords

# Función para determinar la similitud eel texto insertado y el Corpus
def response(user_response):
  robo_response = ''
  sent_tokens.append(user_response) # Añade al Corpus la respuesta de usuario al final
  TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stopwords.words('spanish'))
  TfidfVec = TfidfVec.fit_transform(sent_tokens)

# Evaluar similitud de Coseno entre mensaje usuario (Tfidf[-1]) y el Corpus (Tfidf)
  vals = cosine_similarity(TfidfVec[-1], TfidfVec)
  idx = vals.argsort()[0][-2]
  flat = vals.flatten()
  flat.sort()
  req_tfidf = flat[-2]

  if(req_tfidf == 0):
    robo_response = robo_response + "Lo siento, no te he entendido.Si no puedo responder a lo que buscas póngase en contacto con el número: 0800: 0800-123-4567"
    return robo_response
  else:
    robo_response = robo_response + sent_tokens[idx]
    return robo_response

In [57]:
# Cambia el procesamiento de la función response:
def response(user_response):
    robo_response = ''
    sent_tokens.append(user_response)  # Añade la entrada al final de los tokens
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stopwords.words('spanish'))
    tfidf = TfidfVec.fit_transform(sent_tokens)

    # Evalúa la similitud de coseno y obtiene el índice de la respuesta
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]

    if req_tfidf == 0:
        robo_response += "Lo siento, no te he entendido. Si no puedo responder a lo que buscas, ponte en contacto con el número: 0800-123-4567."
    else:
        robo_response += sent_tokens[idx]
    sent_tokens.pop()  # Elimina la entrada del usuario después de la respuesta
    return robo_response


***Definición de coincidencias manual***

In [58]:
SALUDOS_INPUTS = ("hola", "buenas", "saludos", "qué tal", "hey", "buenos dias", "buenas tardes")
SALUDOS_OUTPUTS = ["Hola", "Hola, ¿Qué tal?", "Hola, ¿Cómo te puedo ayudar?", "Hola, encantado de hablar contigo"]

def saludos(sentence):
  for word in sentence.split():
    if word.lower() in SALUDOS_INPUTS:
      return random.choice(SALUDOS_OUTPUTS)

***Generación de respuestas***

In [59]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
flag = True
print("ROBOT: Mi nombre es Robot. Responderé a tus preguntas acerca de sus vacaciones en el crucero. Si quieres salir, escribe 'salir'")

while flag:
    user_response = input()
    user_response = user_response.lower()  # Convertimos a minúsculas

    if user_response != 'salir':

        if user_response == 'gracias' or user_response == 'muchas gracias':
            flag = True
            print("ROBOT: No hay de qué")
        else:
            if saludos(user_response) is not None:
                print("ROBOT: " + saludos(user_response))
            else:
                print("ROBOT: ", end="")
                print(response(user_response))
    else:
        flag = False
        print("ROBOT: Nos vemos pronto")

ROBOT: Mi nombre es Robot. Responderé a tus preguntas acerca de sus vacaciones en el crucero. Si quieres salir, escribe 'salir'
ROBOT: est prohibido fumar en los teatros, en los restaurantes y buffet de autoservicio (locales cerrados), en los corredores, pasillos, vestbulos, antesalas e interiores de los ascensores y escaleras.
ROBOT: puedes usar el telfono mvil siempre que tengas cobertura suficiente para hacer llamadas, esto suele ser normalmente cuando el buque se encuentra en puerto o navegando cerca de la costa.
